# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686576


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:32,  3.29s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:04,  2.40s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:44,  1.72s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:31,  1.27s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  6.76it/s]

Rendering models:  93%|█████████▎| 28/30 [00:10<00:00,  7.31it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  6.49it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  6.28it/s]

zac8151                               0.000349
Dynamatt                              0.000321
not-logged-in-619dfe56b3aafaf4e45d    0.035577
not-logged-in-75a4c6f7ffb7b2c75141    0.767932
thedarktree                           0.000237
shocko61                              0.000334
not-logged-in-c3e2bb63f9ec3bdeea91    0.001134
mtajnai                               0.000822
not-logged-in-f09819078c043556a4da    0.029114
not-logged-in-7c0e3b1ae8dc7fbb6b66    0.066060
jmhirano                              0.007394
ElisabethB                            0.000334
not-logged-in-6fdf43e5a9997ae450a2    0.001976
not-logged-in-5f30b2bb9b7f9d49da41    0.014232
dxjdxjhhhh                            0.000502
ZannaDragon                           0.000242
JulieHedglin                          0.001842
Susymuller                            0.000457
Lavadude                              0.037990
jordanrushworth                       0.001228
clars915                              0.000416
ts135992     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())